In [1]:
import numpy as np  
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import csv

def load_data(filename):
    x=[]
    target=[]
    a=[]
    with open(filename) as csv_file:
        data_file = csv.reader(csv_file)
        for row in data_file:
            a.append(row)
    print a[0]
    np_a=np.array(a)
    x=np_a[1:,:-1].astype(np.float32)  #read after first row
    target=np_a[1:,-1].astype(np.float32)
    return x, target

In [3]:
X, y=load_data('train.csv')

['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']


In [4]:
print len(X)
print len(y)

8143
8143


In [5]:
X_test, y_test=load_data('test.csv')

['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']


In [16]:
print len(X_test)
print len(y_test)

2665
2665


In [6]:
print X[0]

[  2.31800003e+01   2.72719994e+01   4.26000000e+02   7.21250000e+02
   4.79298783e-03]


In [7]:
#*** Inspect data
n_samples=len(X)
n_features = len(X[0])
print n_samples
print n_features

n_samples_test = len(X_test)
print n_samples_test


8143
5
2665


In [8]:
print y

[ 1.  1.  1. ...,  1.  1.  1.]


In [12]:
# Training
losses = []
losses_test = []
training_steps = 200
learning_rate = 0.002

with tf.Session() as sess:
  # Set up all the tensors, variables, and operations.
  input = tf.constant(X)
  target = tf.constant(np.transpose([y]).astype(np.float32))
  # print target.eval()
  xx = tf.random_normal([5, 1], 0, 0.1)
  weights = tf.Variable(xx)
    
  # print X.max(axis=1)

  tf.initialize_all_variables().run()
  # print(weights.eval())
                      
  yhat = tf.matmul(input, weights)
  
  #yhatS = tf.sigmoid(yhat)
    
  # yerror = tf.sub(yhat, target)
  #loss = tf.nn.l2_loss(yerror)
  #loss = tf.nn.softmax(yhat,yerror)
  loss = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(yhat, target) )
                                                
  update_weights = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  for _ in range(training_steps):
    update_weights.run()
    losses.append(loss.eval())
    # print _  #It takes on value from 0 to 49
    # print losses  #this shows losses array keep increasing in size: [18] , [18, 13],...
    # print losses[-1]
    # print loss.eval()

  # Training is done, get the final values for the graphs
  betas = weights.eval()
  yhat = yhat.eval()
  print betas

  # Testing
  input_test = tf.constant(X_test)
  target_test = tf.constant(np.transpose([y_test]).astype(np.float32))
  # print target.eval()
  weights = tf.Variable(betas)

  # bestBetas = np.transpose([-0.56908154, -0.43243277, 0.38995218, -0.05960743, 0.01064102]);
  # print bestBetas
  # weights = tf.Variable(bestBetas);
    
  yhat_test = tf.matmul(input_test, weights)
  # print X.max(axis=1) 
    
  yerror = tf.sub(yhat_test, target_test)

  tf.initialize_all_variables().run()
    
  print weights.eval()
  # print yhat_test.eval()
  # print target_test.eval()
  loss = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(yhat_test, target_test) )
  # print loss.eval()
    
  yhat_testS = tf.sigmoid( yhat_test )
    
  TP = 0
  FP = 0
  FN = 0
  TN = 0
#  for i in range(0, len(y_test)):
#     print '{}: obs: {} pred:{}'.format(i, target_test.eval()[i], yhat_testS.eval()[i] )
   
  for i in range(0, len(y_test)):
    if target_test.eval()[i]==1:
      if yhat_testS.eval()[i]>0.5:
        TP = TP + 1;
      else:
        FN = FN + 1;    
        
    if target_test.eval()[i]==0:
      if yhat_testS.eval()[i]>0.5:
        FP = FP + 1;
      else:
        TN = TN + 1;
                

 #      
 #    if target_test.eval()[i]==1 & yhat_testS.eval()[i]>0.5:
 #      TP = TP + 1;
 #    elif target_test.eval()[i]==0 & yhat_testS.eval()[i]<0.5:
 #      TN = TN + 1;
 #    elif target_test.eval()[i]==1 & yhat_testS.eval()[i]<0.5:
 #      FN = FN + 1;
 #    elif target_test.eval()[i]==0 & yhat_testS.eval()[i]>0.5:
 #      FP = FP + 1;
 
  accuracy = float((TP+TN))/len(y_test)
  sensitivity = float(TP)/(TP+FN)
  specificity = float(TN)/(TN+FP)
  print 'Accuracy = {}%'.format(accuracy*100.) 
  print 'Sensitivity = {}%'.format(sensitivity*100.) 
  print 'Specificity = {}%'.format(specificity*100.) 
    

  



[[-0.83215612]
 [-0.33642992]
 [ 0.32360554]
 [-0.46921009]
 [ 0.00595133]]
[[-0.83215612]
 [-0.33642992]
 [ 0.32360554]
 [-0.46921009]
 [ 0.00595133]]
Accuracy = 63.564727955%
Sensitivity = 0.102880658436%
Specificity = 100.0%
